# Intro to Signal Processing

In [ ]:
%load_ext autoreload
%autoreload 2

import csv
from biosignals.eog import EOG
import matplotlib.pyplot as plt
import numpy as np

Here, we have EEG data from the frontal lobe of a person meditating using the Muse headband. Try graphing the data; what do you notice?

In [ ]:
# Saving Muse EEG values in list: one for each electrode
l_ear_arr = []
l_forehead_arr = []
r_forehead_arr = []
r_ear_arr = []

with open('./data/stare_blink.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        l_ear = float(row['l_ear'])
        l_forehead = float(row['l_forehead'])
        r_forehead = float(row['r_forehead'])
        r_ear = float(row['r_ear'])
        
        l_ear_arr.append(l_ear)
        l_forehead_arr.append(l_forehead)
        r_forehead_arr.append(r_forehead)
        r_ear_arr.append(r_ear)

The signal looks really noisy! This is because there are many sources of noise that affect the EEG reading we collected, from our skin not being a good conductor to the electric fields of literally every electrical appliance in the room! Let's try cleaning up the data!

Note that the EEG we are interested in analyzing rarely goes above 40Hz, so we can just remove any signal above 40Hz! Try doing that below:

That looks pretty good, but we can do better! Note that the 0Hz component is the same as a sine wave with a period of 1/0 seconds ... pretty much a straight line! Essentially the 0Hz component shifts the graph up and down. We really don't care about that either ... so let's try removing it!